# Modèle QANet pour du *Question Answering*
Base de données: https://rajpurkar.github.io/SQuAD-explorer/

## Importation des données

In [31]:
import pandas as pd
df = pd.read_excel('./questions_scolarité.xlsx')
pd.DataFrame.dropna(df, axis='index', how='any', thresh=3, inplace=True) # on veut au plus 0 erreurs "NaN" par ligne
print(df.loc[:5,['question','contexte']].to_latex(index=False,caption=("5 premières phrases du dataset constitué avec les documents de la scolarité","Questions et réponses du dataset de la scolarité")))

\begin{table}
\centering
\caption[Questions et réponses du dataset de la scolarité]{5 premières phrases du dataset constitué avec les documents de la scolarité}
\begin{tabular}{ll}
\toprule
                                          question &                                           contexte \\
\midrule
          Qui approuve le règlement de scolarité ? & Le Règlement de Scolarité est approuvé par le C... \\
Par quelle voie s'effectue l'admission en premi... & L'admission en 1ère année de la formation ingén... \\
Comment est l'inscription pour les élèves en pr... & Pour les élèves ayant signé un contrat de profe... \\
 Que les élèves doivent-ils avoir lu et approuvé ? & Les élèves ont lu, approuvé et signé les charte... \\
  Pour quoi la formation peut-elle être aménagée ? & La formation peut être aménagée pour effectuer ... \\
            Qu'est-ce qu'une action de formation ? & Une Action de Formation est un ensemble cohéren... \\
\bottomrule
\end{tabular}
\end{table}



In [28]:
df.truncate(before=0, after=5)

,question,contexte,reponse
0,Qui approuve le règlement de scolarité ?,Le Règlement de Scolarité est approuvé par le ...,Conseil d'Administration de l'École
1,Par quelle voie s'effectue l'admission en prem...,L'admission en 1ère année de la formation ingé...,concours commun « Centrale-Supélec »
2,Comment est l'inscription pour les élèves en p...,Pour les élèves ayant signé un contrat de prof...,uniquement pédagogique
3,Que les élèves doivent-ils avoir lu et approuvé ?,"Les élèves ont lu, approuvé et signé les chart...",les chartes informatique et anti-plagiat
4,Pour quoi la formation peut-elle être aménagée ?,La formation peut être aménagée pour effectuer...,pour effectuer un double diplôme
5,Qu'est-ce qu'une action de formation ?,Une Action de Formation est un ensemble cohére...,un ensemble cohérent d'activités planifiées


In [52]:
l_data = []
for d in df.itertuples():
    l_data.append([d[1],d[2],d[3]])
l_data[:3]

[['Qui approuve le règlement de scolarité ?',
  "Le Règlement de Scolarité est approuvé par le Conseil d'Administration de l'École après consultation du Conseil des Études.",
  "Conseil d'Administration de l'École"],
 ["Par quelle voie s'effectue l'admission en première année ?",
  "L'admission en 1ère année de la formation ingénieur s'effectue par la voie du concours commun « Centrale-Supélec » et par la voie de l'admission sur titre, soit « CASTing », admission sur titres d'ingénieurs, commun au Groupe des Écoles Centrales, soit « ENIS ».",
  'concours commun « Centrale-Supélec »'],
 ["Comment est l'inscription pour les élèves en profesionnalisation ?",
  "Pour les élèves ayant signé un contrat de professionnalisation, l'inscription est uniquement pédagogique, c'est-à-dire sans règlement de Droits Universitaires réglementaires.",
  'uniquement pédagogique']]

## Acqisition des données

### Algorithme KMP pour la création du *dataset*

In [31]:
# Knuth-Morris-Pratt string matching
# David Eppstein, UC Irvine, 1 Mar 2002
# trouvé d'après https://stackoverflow.com/questions/425604/best-way-to-determine-if-a-sequence-is-in-another-sequence
# code original ici: https://code.activestate.com/recipes/117214/
from __future__ import generators

def KnuthMorrisPratt(text, pattern):

    '''Yields all starting positions of copies of the pattern in the text.
Calling conventions are similar to string.find, but its arguments can be
lists or iterators, not just strings, it returns all matches, not just
the first one, and it does not need the whole text in memory at once.
Whenever it yields, it will have read the text exactly up to and including
the match that caused the yield.'''

    # allow indexing into pattern and protect against change during yield
    pattern = list(pattern)

    # build table of shift amounts
    shifts = [1] * (len(pattern) + 1)
    shift = 1
    for pos in range(len(pattern)):
        while shift <= pos and pattern[pos] != pattern[pos-shift]:
            shift += shifts[pos-shift]
        shifts[pos+1] = shift

    # do the actual search
    startPos = 0
    matchLen = 0
    for c in text:
        while matchLen == len(pattern) or \
              matchLen >= 0 and pattern[matchLen] != c:
            startPos += shifts[matchLen]
            matchLen -= shifts[matchLen]
        matchLen += 1
        if matchLen == len(pattern):
            yield startPos


# https://stackoverflow.com/questions/17870544/find-starting-and-ending-indices-of-sublist-in-list
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll-1

Tester notre algo de KMP pour le comptage des indices

In [20]:
list(KnuthMorrisPratt([1,2,3,4,5,6,1,2,3,4],[2,3,4])) # un deux match en position 1 et 7

[1, 7]

### Extraction des variables de texte et des indices des réponses à l'aides d'outils de NLP et création de la liste Dataset

In [62]:
import IPython.display
import re as regex
import spacy

nlp = spacy.load('fr_core_news_sm')

def traitement_nlp(text):
    ctxt = regex.sub(r'[^0-9 ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzäëïöüâêîôûàéêèçùÉÀÈÇÂÊÎÔÛÄËÏÖÜÀÇÉÈÙ,:.?!;\[\]\(\)\'-]','', text) # traitement ici pour optimiser
    ctxt = regex.sub(r"\[[^\[\]]*\]",'',ctxt)   # enlever les brackets et leur contenu
    ctxt = regex.sub(r'[" "]+', " ", ctxt)      # remplacer les surplus d'espaces par uniquement un espace
    ll = [tok.text.lower() for tok in nlp(ctxt)]        # la phrase complète en liste de strings (tokens)
    ctxt = ' '.join(ll)                         # la phrase complète en string dont les tokens sont séparés par des espaces
    return ctxt,ll



# on crée ensuite une liste dont les éléments sont des listes comprenant 5 éléments dans l'ordre contexte, question, réponse, indice, indice
dataset = []
for datapoint in l_data:
    ctxt,ctxt_tok    = traitement_nlp(datapoint[1]) # on récupère le contexte en string et liste de tokens strings
    quest,quest_tok  = traitement_nlp(datapoint[0]) # on récupère la question en string et liste de tokens strings
    answ,answ_tok    = traitement_nlp(datapoint[2]) # on récupère la liste des tokens de la question pour la matcher avec celle du contexte
    ind_match = list(KnuthMorrisPratt(ctxt_tok,answ_tok))
    try:
        answ_start = ind_match[0]
        answ_end = ind_match[0] + len(answ_tok) # convention intervalle semi ouvert
        dataset.append([ctxt, quest, answ, answ_start, answ_end]) # str,str,str,int,int
    except IndexError: # mauvaise tokénisation de la réponse ou erreur de typo dans le texte ou modification destructive due au prétraitement
        pass


In [63]:
dataset[0]

["le règlement de scolarité est approuvé par le conseil d' administration de l' école après consultation du conseil des études .",
 'qui approuve le règlement de scolarité ?',
 "conseil d' administration de l' école",
 8,
 14]

#### Sauvegarder la liste du dataset

In [58]:
import pickle
with open("./questions_réponses_scol", "wb") as fp:
    pickle.dump(dataset, fp)